In [1]:
import torch
from interactive import Generator
import sentencepiece as spm
import fileinput

In [2]:
### Create SentencePiecer ###
m=spm.SentencePieceProcessor(model_file='/home/bhaddow/experiments/e2e-slt-noise/data/baseline-mt/en-fr/spm_unigram8000_en.model')

In [3]:
### Test Inputs ###
INPUT='Hello my friends welcome to your new home'.split()
INPUT = 'Hello my friends. Welcome to your new home!'.split()
LENGTH=len(INPUT)

In [4]:
### Input from file ###
def buffered_read(input, buffer_size):
        buffer = []
        #here the input for interactive is read, the fileinput is a python class. The input method takes a file or if passed '-' will read from sys.stin
        with fileinput.input(files=[input], openhook=fileinput.hook_encoded("utf-8")) as h: 
            for src_str in h:
                buffer.append(src_str.strip())
                if len(buffer) >= buffer_size:
                    yield buffer
                    buffer = []

        if len(buffer) > 0:
            yield buffer

In [5]:
def RANDOM_POLICY():
    return 1 if torch.rand(1).item()>0.5 else 0

def UPDATE_COUNTER(COUNTER):
    if COUNTER==0:
        return COUNTER+1,"START"
    else:
        # GET ACTION
        ACTION=RANDOM_POLICY()  
        if ACTION==0:
            return COUNTER+1,"R"
        else:
            return COUNTER,"W"

def GET_STREAM(COUNTER, INPUT, LENGTH):
    if COUNTER==LENGTH: # Need to ensure that I am translating everything after the END
        return " ".join(INPUT),"END",COUNTER
    else:
        COUNTER,ACTION=UPDATE_COUNTER(COUNTER)
        return " ".join(INPUT[:COUNTER]),ACTION,COUNTER

def ACT(COUNTER, INPUT, LENGTH):
    STREAM,ACTION,COUNTER=GET_STREAM(COUNTER, INPUT, LENGTH)
    # Print ACTION
    if ACTION=="END":
        print("...END...")
    elif ACTION=="START":
        print("...START...")
    elif ACTION=="R":
        print("...READ...")
    else:
        print("...WRITE...")
    # Display NMT input
    print(STREAM)
    # Covert to sentence piece
    STREAM=m.encode(STREAM, out_type=str)
    return " ".join(STREAM),COUNTER,ACTION

In [6]:
# Perform translations
from interactive import Generator
NMT = Generator("/home/bhaddow/experiments/e2e-slt-noise/data/baseline-mt/en-fr/data-bin", "/home/bhaddow/experiments/e2e-slt-noise/expts/baseline-mt/en-fr/checkpoints/checkpoint_best.pt")
#STATEMENT1 = NMT.generate(" ".join(m.encode('Start', out_type=str)))
#STATEMENT2 = NMT.generate(" ".join(m.encode('New line', out_type=str)))
#STATEMENT3 = NMT.generate(" ".join(m.encode('Done', out_type=str)))
STATEMENT1 = "STARTING TRANSLATION"
STATEMENT2 = "COMMENCING NEW LINE"
STATEMENT3 = "FINISHED"

with open('target_stream.txt', 'a') as f:
    f.write(STATEMENT1 + '...' + '\n')

i=0

for INPUT in buffered_read('source.txt', 0):
    COUNTER=0
    ACTION=None
    INPUT=INPUT[0].split()
    LENGTH=len(INPUT)
    
    if i:
        with open('target_stream.txt', 'a') as ts:
            ts.write(STATEMENT2 + '...' + '\n')
        
    else:
        i=1

    while ACTION!="END":
        STREAM,COUNTER,ACTION=ACT(COUNTER,INPUT,LENGTH)
        # Either Read or Write
        if ACTION=="R" or ACTION=="START":
            continue
        else:
            # Perform translation via NMT model
            #print(f"the stream: {STREAM}")
            OUTPUT = NMT.generate(STREAM)
            # Display the output
            print(OUTPUT)
            # Put the stream into a file
            with open('target_stream.txt', 'a') as ts, open('target.txt', 'a') as t:
                if ACTION=="W":
                    ts.write(OUTPUT + '...' + '\n')
                elif ACTION=="END":
                    ts.write(OUTPUT + '\n')
                    t.write(OUTPUT + '\n')

    
with open('target_stream.txt', 'a') as ts:
    ts.write(STATEMENT3 + '\n')





<class 'fairseq.models.transformer.transformer_decoder.TransformerDecoderBase'>
...START...
Hello
...WRITE...
Hello
the tokens: tensor([[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
the tokens: tensor([[   2, 4514,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1

NotImplementedError: 